# Combine BioASQ with Our IR

In [108]:
import pandas as pd
import os
import json
import torch
import time
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

import json
import time
from tqdm import tqdm  # Import tqdm for the progress bar

## Setting for queries

In [109]:
host = '3.23.103.76' #host = 'localhost' 
port = 9200
auth =('admin','IVIngi2024!') #auth = ('admin','admin') 
client_lexical = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    timeout=500, 
    max_retries=1
    #connection_class=RequestsHttpConnection 
#    http_compress = True, # enables gzip compression for request bodies
#    use_ssl = False,
#   verify_certs = False,
#    ssl_assert_hostname = False,
#    ssl_show_warn = False
)
print("Connection opened...")

Connection opened...


In [110]:

query_text={
  "query": {
    "match": {
      "pmid": "15858239"
    }
  },
  "_source": ["text"]  
}

# Execute the query
response = client_lexical.search(
    index="medline-faiss-hnsw-lexical-pmid",
    body=query_text
)
print(response)

{'took': 1, 'timed_out': False, '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 15.262341, 'hits': [{'_index': 'medline-faiss-hnsw-lexical-pmid', '_id': '15858239', '_score': 15.262341, '_source': {}}]}}


## Prepare data for generarive component

In [47]:
#Get text from our IR for given pmid

    
def get_document_text(pmid, client):
    # Format the query text with the correct PubMed ID
    query_text = {
        "query": {
            "match": {
                "pmid": pmid
            }
        },
        "_source": ["full_text"]
    }

    # Use the provided OpenSearch client to execute the query
    response = client_lexical.search(
        index="medline-faiss-hnsw-lexical-pmid",
        body=query_text
    )

    # Check if the response has hits and return the cleaned text
    if response['hits']['hits']:
        # Extract the full text from the first hit
        full_text = response['hits']['hits'][0]['_source']['full_text']
        # Clean the text by removing new lines and excess white spaces
        cleaned_text = ' '.join(full_text.replace('\n', ' ').split())
        return cleaned_text
    else:
        # Return a default message or handle as needed
        return f"No document found for PMID: {pmid}"
    


In [48]:
#checking one example
pmid = "26422722"  # Example PubMed ID
document_text = get_document_text(pmid, client_lexical)
print(document_text)

Phase 3 Studies Comparing Brodalumab with Ustekinumab in Psoriasis. BACKGROUND Early clinical studies suggested that the anti-interleukin-17 receptor A monoclonal antibody brodalumab has efficacy in the treatment of psoriasis. METHODS In two phase 3 studies (AMAGINE-2 and AMAGINE-3), patients with moderate-to-severe psoriasis were randomly assigned to receive brodalumab (210 mg or 140 mg every 2 weeks), ustekinumab (45 mg for patients with a body weight ≤100 kg and 90 mg for patients >100 kg), or placebo. At week 12, patients receiving brodalumab were randomly assigned again to receive a brodalumab maintenance dose of 210 mg every 2 weeks or 140 mg every 2 weeks, every 4 weeks, or every 8 weeks; patients receiving ustekinumab continued to receive ustekinumab every 12 weeks, and patients receiving placebo received 210 mg of brodalumab every 2 weeks. The primary aims were to evaluate the superiority of brodalumab over placebo at week 12 with respect to at least a 75% reduction in the pso

In [49]:
json_structure = {
    "questions": [
        {
            "question": "Question here",
            "type": "summary",
            "exact_answer": ["Exact answer here"],
            "ideal_answer": "Ideal answer here",
            "abstracts": []
        }
    ]
}

In [50]:
"""def populate_abstracts(question, pmids, client):
   
    #Populate the 'abstracts' field for a given question with texts fetched using PubMed IDs. 
    #Args:
    #question (dict): The question dictionary to populate.
    #pmids (list of str): List of PubMed IDs.
    #client (OpenSearchClient): The OpenSearch client instance.
   
    # Clear existing abstracts (if this function is to be reused)
    question['abstracts'] = []

    for pmid in tqdm(pmids, desc="Fetching texts", leave=False):
        text = get_document_text(pmid, client)
        question['abstracts'].append({
            "id": pmid,
            "text": text
        })
"""
def populate_abstracts(question, pmids, client):
    # Initialize the list with None to reserve order
    abstracts = [None] * len(pmids)

    # Using tqdm to show progress for fetching document texts
    for index, pmid in enumerate(tqdm(pmids, desc="Fetching texts", leave=False)):
        text = get_document_text(pmid, client)
        abstracts[index] = {
            "id": pmid,
            "text": text
        }

    # Filter out None values in case any text could not be fetched
    question['abstracts'] = [abstract for abstract in abstracts if abstract]


In [51]:
#example for populating abstract
question = {"question": "What are the latest treatments for diabetes?"}
pmids = ['26422722']  # Example PMIDs

# Populate abstracts for the question
populate_abstracts(question, pmids, client_lexical)

# Printing out the populated question to see the result
print(question)

{'question': 'What are the latest treatments for diabetes?', 'abstracts': [{'id': '26422722', 'text': "Phase 3 Studies Comparing Brodalumab with Ustekinumab in Psoriasis. BACKGROUND Early clinical studies suggested that the anti-interleukin-17 receptor A monoclonal antibody brodalumab has efficacy in the treatment of psoriasis. METHODS In two phase 3 studies (AMAGINE-2 and AMAGINE-3), patients with moderate-to-severe psoriasis were randomly assigned to receive brodalumab (210 mg or 140 mg every 2 weeks), ustekinumab (45 mg for patients with a body weight ≤100 kg and 90 mg for patients >100 kg), or placebo. At week 12, patients receiving brodalumab were randomly assigned again to receive a brodalumab maintenance dose of 210 mg every 2 weeks or 140 mg every 2 weeks, every 4 weeks, or every 8 weeks; patients receiving ustekinumab continued to receive ustekinumab every 12 weeks, and patients receiving placebo received 210 mg of brodalumab every 2 weeks. The primary aims were to evaluate th

In [52]:

import json
#evaluation_file = 'training12b_new.json'
#with open(evaluation_file, 'r') as f:
 #   data = json.load(f)

#print(len(data['questions']))
#BioASQ_filepath = 'training12b_new.json'
#BioASQ_filepath = 'BioASQ_small.json'
BioASQ_filepath = 'BioASQ_10_abstracts.json'
def read_bioasq_file(filepath):
    """
    Read the BioASQ JSON file.
    
    Args:
    filepath (str): The path to the BioASQ file.
    
    Returns:
    list: A list of questions from the BioASQ file.
    """
    with open(filepath, 'r') as file:
        bioasq_data = json.load(file)
    return bioasq_data['questions']

In [53]:


def create_custom_json_structure(bioasq_questions, client):
    """
    Create a custom JSON structure based on BioASQ data.
    
    Args:
    bioasq_questions (list): List of questions from BioASQ data.
    client (OpenSearchClient): The OpenSearch client instance.
    
    Returns:
    dict: The custom JSON structure populated with abstracts.
    """
    json_structure = {"questions": []}

    for question in tqdm(bioasq_questions, desc="Processing questions"):
        # Create a new question entry
        new_question = {
            "id": question.get("id", "No ID"), 
            "question": question.get("body", "No question body"),
            "type": question.get("type", ""),
            "exact_answer": question.get("exact_answer", []),
            "ideal_answer": question.get("ideal_answer", "No ideal answer"),
            "abstracts": []
        }

        # Extract PubMed IDs from the 'documents' URLs
        pmids = [doc.split('/')[-1] for doc in question.get('documents', [])]
        
        # Populate abstracts
        populate_abstracts(new_question, pmids, client)
        
        # Add the populated question to the json structure
        json_structure['questions'].append(new_question)

    return json_structure

## Create custom BioASQ-ALL - with abstract's text from our IR for pmid from bioASQ

In [17]:
# Example usage
start_time = time.time()

bioasq_questions = read_bioasq_file(BioASQ_filepath)
custom_json = create_custom_json_structure(bioasq_questions, client_lexical)

end_time = time.time()
elapsed_time = end_time - start_time
# Save the new JSON structure to a file
with open('Generative/custom_bioasq_output_10_abstracts_without_newline.json', 'w') as outfile:
    json.dump(custom_json, outfile, indent=4)

print(f"Time taken to create the custom JSON: {elapsed_time:.2f} seconds")

Processing questions: 100%|██████████| 182/182 [04:49<00:00,  1.59s/it]


Time taken to create the custom JSON: 290.20 seconds


# Create subset of BioASQ - question with 10 relevant abstract

In [54]:

def filter_questions_with_ten_or_more_documents(input_filepath, output_filepath):
    # Load the data from the BioASQ JSON file
    with open(input_filepath, 'r') as file:
        data = json.load(file)
    
    filtered_questions = []

    # Filter questions based on the number of documents
    for question in data['questions']:
        # Check if there are 10 or more documents
        if len(question['documents']) == 10:
            filtered_questions.append(question)

    # Prepare the final structure with filtered questions
    final_data = {'questions': filtered_questions}
    # Save the filtered data to a new JSON file
    with open(output_filepath, 'w') as outfile:
        json.dump(final_data, outfile, indent=4)

    print(f"Filtered data has been saved to {output_filepath}")
    print("Number od questions with 10 or more abstracts:", len(filtered_questions))
# Define file paths
input_filepath = 'training12b_new.json'  # Set the correct path to your BioASQ file
output_filepath = 'BioASQ_10_abstracts.json'

# Process the file
filter_questions_with_ten_or_more_documents(input_filepath, output_filepath)


Filtered data has been saved to BioASQ_10_abstracts.json
Number od questions with 10 or more abstracts: 182


### <font color='red'>Kraj pripreme podataka za generativnu komponentu</font>

In [111]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
from qdrant_client.http import models


# 3.145.52.195
#client_semantic = QdrantClient(host, port=6333, timeout = 60)

verifai_ip='3.23.103.76'
qdrant_port=6333
TIMEOUT=60
url = f"https://{verifai_ip}:{qdrant_port}"
qdrant_api="8da7725d78141e19a9bf3d878f4cb333fedb56eed9727904b46ce4b32e1ce085"
client_semantic = QdrantClient(url=url, api_key=qdrant_api, timeout=TIMEOUT, https=True,**{'verify': False})
#client_semantic = QdrantClient(host, port=6333, timeout = 60)

# Select the type of lexical indexing: 

In [112]:
lexical_pmid = False

if lexical_pmid:
    index_name_lexical = 'medline-faiss-hnsw-lexical-pmid'
else:
    index_name_lexical ='medline-faiss-hnsw-lexical'

coll_name_semantic = "medline-faiss-hnsw"

In [113]:
model_card = 'sentence-transformers/msmarco-distilbert-base-tas-b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device {device}")

Device cpu


In [114]:
model = SentenceTransformer(model_card)

c:\Users\Adela\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Query

In [115]:
import nltk

# Ensure that the necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd




class QueryProcessor:
    def __init__(self, index_lexical:str = "medline-faiss-hnsw-lexical",lexical_pmid = False, index_name_semantic ="medline-faiss-hnsw", rescore = False, model=None, lexical_client=None, semantic_client=None, output_file_path="queries/queries.tsv", stopwords=set([])):
        self.index_lexical_name = index_lexical
        self.index_name_semantic = index_name_semantic
        # 2 index name (?)
        self.model = model
        #self.lexical_pmid = lexical_pmid
        self.lexical_client = lexical_client
        self.semantic_client = semantic_client
        self.output_file_path = output_file_path
        self.stop_words = stopwords
        self.query_result = []
        self.rescore = rescore
        self.lexical_query = self.lexical_query_pmid if lexical_pmid else self.lexical_query
    
    def set_rescore(self, rescore):
        self.rescore = rescore

    def preprocess_query(self, query_str):
        return ' '.join([word for word in word_tokenize(query_str) if word.lower() not in self.stop_words])

    def save_results(self):
        with open(self.output_file_path, "w") as file:
            json.dump(self.query_result, file, indent=4)
      
    
    def reorder_pmid(self, retrived_documents):
        pmid_scores = {}
        
        # Iterate through the set data
        for _, value in retrived_documents.items():
            pmid = value['pmid']
            score = value['score']
            
            # Check if pmid already exists in the dictionary
            if pmid in pmid_scores:
                pmid_scores[pmid] += score
            else:
                pmid_scores[pmid] = score
           
        return pmid_scores
    
    def lexical_query(self, query_str, limit=10):
        if self.lexical_client == None:
            raise ValueError("No Lexical client defined")
        
        query = {
                "size": limit,
                "query": {
                    "multi_match": {
                        "query": query_str,
                        "fields": ["text"]
                    }
                }
            }
       
        results = self.lexical_client.search(index=self.index_lexical_name, body=query) 
        retrived_documents = {}
        max_score = results['hits']['max_score']
     
        for hit in results["hits"]["hits"]:
            
            pmid = hit["_source"]["pmid"]
            score = hit["_score"]
            auto_id = hit["_id"]
            
            
            
            retrived_documents[auto_id] = {
                "score": round(score/max_score, 5),
                "pmid": pmid
                }
        
        retrived_documents = self.reorder_pmid(retrived_documents)
        return retrived_documents #adjust the return 
    
    def lexical_query_pmid(self, query_str, limit=10):
        #print("Lexical = ",query_str)
        if self.lexical_client == None:
            raise ValueError("No Lexical client defined")
        
        query = {
                "size": limit,
                "query": {
                    "multi_match": {
                        "query": query_str,
                        "fields": ["full_text"]
                    }
                }
            }
        
        results = self.lexical_client.search(index=self.index_lexical_name, body=query) 
        
        retrieved_documents = {}
        max_score = results['hits']['max_score']
        for hit in results["hits"]["hits"]:
            
            pmid = hit["_source"]["pmid"]
            score = hit["_score"] / max_score
            
            retrieved_documents[pmid] = score
            
        return retrieved_documents #adjust the return 

    def semantic_query(self, query, limit=10):
        #print("semantic = ",query)
        if self.semantic_client == None:
            raise ValueError("No Semantic client defined")
        if self.model == None:
            raise ValueError("No model defined")
        
        query_vector = self.model.encode(query).tolist()
    
        search_params=models.SearchParams(
            quantization=models.QuantizationSearchParams(rescore=self.rescore)
            )
        results = self.semantic_client.search(collection_name=self.index_name_semantic,query_vector=query_vector,search_params=search_params, limit=limit)
    
        #results = self.semantic_client.search(collection_name=self.index_name_semantic,query_vector=query_vector, limit=limit)
        
        retrived_documents = {}
        max_score = None
        for i,document in enumerate(results):
            
            pmid = document.payload['pmid']
            score = document.score
            if i == 0:
                # first score is the max
                max_score = score
            retrived_documents[document.id] = { 'pmid': pmid, 'score': round(score / max_score, 5) } 

        retrived_documents = self.reorder_pmid(retrived_documents)
        
        return retrived_documents
    

    def hybrid_query(self, query_lexical, query_semantic, lex_parameter = 0.5, semantic_parameter = 0.5, limit=10):
        if (lex_parameter + semantic_parameter) > 1:
            raise ValueError("Uncorrect parameters for Hybrid Queries")
        lexical_results = self.lexical_query(query_lexical, limit = limit) 
        semantic_results = self.semantic_query(query_semantic, limit)
        max_score = 0
        retrived_documents = {}
        
        for lex_pmid in lexical_results:
            score = lexical_results[lex_pmid] * lex_parameter
            if lex_pmid in semantic_results:
                score += semantic_results[lex_pmid] * semantic_parameter

            retrived_documents[lex_pmid] = score
            max_score = max(max_score, score)
            

        for semantic_pmid in semantic_results:
            if semantic_pmid not in lexical_results:
                score = semantic_results[semantic_pmid] * semantic_parameter
                retrived_documents[semantic_pmid] = score
                max_score = max(max_score, score)
                
        return retrived_documents # just to have a starting point

    def execute_query(self, query_str, query_type='lexical', lex_parameter = 0.5, semantic_parameter = 0.5,limit = 10,save = True, stopwords_preprocessing=True):
        #print("Before = ",query_str)
        text_query = self.preprocess_query(query_str) if stopwords_preprocessing else query_str
        
        if query_type == 'lexical':
            results = self.lexical_query(text_query, limit=limit) 
        
        elif query_type == 'semantic':
            results = self.semantic_query(query_str, limit=limit)
            
    #The final list of documents is sorted by the combined score, allowing the system to return a balanced mix of results that consider both lexical and semantic relevance.
        elif query_type == 'hybrid':
            results = self.hybrid_query(text_query, query_str, lex_parameter, semantic_parameter, limit=limit)
        else:
            raise ValueError("Invalid query type specified. Choose 'lexical', 'semantic', or 'hybrid'.")
        
        
        document_retrived = sorted(results.items(), key=lambda x: x[1], reverse=True)
        document_retrived = document_retrived[:limit+1] # in the hybrid search we can return more documents
        #print("Results ", document_retrived)
        if save:
            self.process_results(document_retrived, query_str, query_type)

        return document_retrived
    
    # needs to be rewrited
    def process_results(self, results, query_str,query_type):
        
        retrieved_documents = []
        for element in results:
            
            pmid,_ = element
            query = {
                    "query": {
                        "term": {
                        "pmid": int(pmid)
                        }
                    }
                }

            results = self.lexical_client.search(index=self.index_lexical_name, body=query) 
            full_text = results['hits']['hits'][0]["_source"]['full_text']
            pmid = results['hits']['hits'][0]["_source"]['pmid']

            retrieved_documents.append({
                "pmid": pmid,
                "text": full_text
            })

        dict_to_save = {'query': query_str, 'query_type': query_type, 'abstracts' : retrieved_documents}
        self.query_result.append(dict_to_save)  

    


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [116]:
#known_pmid = '15795476'
qp = QueryProcessor(
        model=model,  # You need to define or load your model
        lexical_client=client_lexical,  # Opensearch client 
        semantic_client=client_semantic  # Qdrant client
    )
print(qp.execute_query("What is the cause of hip pain",query_type='hybrid',lex_parameter=0.7, semantic_parameter=0.3,limit=10))
#print(qp.hybrid_query(client_lexical,client_semantic,lex_parameter=0.7,semantic_parameter=0.3,limit=10))

[('31431276', 0.7), ('1570146', 0.6332199999999999), ('26692249', 0.630973), ('30509429', 0.6260939999999999), ('37751184', 0.620501), ('37777849', 0.620501), ('33132093', 0.607124), ('31272626', 0.590828), ('38090918', 0.589488), ('36621448', 0.588007), ('1622374', 0.5859139999999999)]


## Query parser


Doing some queries

In [117]:
model = SentenceTransformer(model_card)

In [118]:
query_parser = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic)

In [119]:
query_parser.execute_query(query_str="Which gene is responsible for disfunction in speech for children?", query_type='hybrid')

[('10797929', 0.5),
 ('15877281', 0.5),
 ('37560515', 0.49687),
 ('29597095', 0.494725),
 ('20870045', 0.49326),
 ('29922639', 0.49277),
 ('9462748', 0.490925),
 ('22303795', 0.49066),
 ('24914010', 0.490565),
 ('19332160', 0.49053),
 ('22106036', 0.49053)]

## Evaluation

## Loading the evaluation file - BioASQ_10_or_more.json

In [120]:
import json
evaluation_file = 'BioASQ_10_abstracts.json'

with open(evaluation_file, 'r') as f:
    data = json.load(f)

print(len(data['questions']))

182


## Evaluation Function

In [131]:
import numpy as np


def clean_documents(documents):
    output_documents = []  ## Changed from output_documents = set()
    for doc in documents:
        output_documents.append((doc.replace("http://www.ncbi.nlm.nih.gov/pubmed/","")))
    return output_documents

def average_precision(retrived_doc, true_doc):
    # Initialize variables
    precision_sum = 0
    num_retrieved_docs = 0
    
    # Calculate precision at each relevant document position
    for i, retrived in enumerate(retrived_doc, start=1):
        pmid,_ = retrived
        if pmid in true_doc:  # Check if the document is relevant
            num_retrieved_docs += 1
            precision_sum += num_retrieved_docs / i  # Calculate precision at cutoff i

    # Calculate average precision
    if num_retrieved_docs == 0:
        return 0
    else:
        return precision_sum / num_retrieved_docs


def evaluation(query_parser, data, query_type,alpha=0.5, beta=0.5, stopwords_preprocessing = False, path = "query_result.json"):
    avg_precisions_sum = [] # sum all average precision and divide with number of queries 
    precisions_sum = []
    queries_time = []
    for i,question in enumerate(data['questions']):
        dict_to_save = {}
        query = question['body']
        type_question = question['type']
        dict_to_save['query'] = query
        dict_to_save['query_type'] = query_type
        dict_to_save['question_type']=type_question # ADDED LATER FOR THE PURPOSE OF STATISTIC
        #relevant_documents = clean_documents(question['documents'])
        #(NEW_EVAL)Return only the first 10 abstracts
        relevant_documents = clean_documents(question['documents'])[:10]
        start_time = time.time()
        
        #results = query_parser.execute_query(query,query_type = query_type, lex_parameter = alpha, semantic_parameter = beta,limit = len(relevant_documents), save=False, stopwords_preprocessing = stopwords_preprocessing)
       
        #(NEW_EVAL)In the next line, limit is set to 10 for retrieving from our IR
        results = query_parser.execute_query(query,query_type = query_type, lex_parameter = alpha, semantic_parameter = beta,limit = 10, save=False, stopwords_preprocessing = stopwords_preprocessing)

        queries_time.append(time.time() - start_time)
        
        #results = [('20598273',1), ('4',1), ('6650562',1), ('2',1),('21995290',1),('15617541',1),('23001136',1),('8896569',1), ('12239580',1)]
        dict_to_save['true_documents'] = relevant_documents
        dict_to_save['retrieved_documents'] = results
       

    
        number_retrieved_documents = 0
        for pmid,_ in results:
            if pmid in relevant_documents:
                number_retrieved_documents +=1

        precision = number_retrieved_documents / len(relevant_documents)
        recall = number_retrieved_documents / len(relevant_documents)
        avg_precision = average_precision(results, relevant_documents)
        
        precisions_sum.append(precision)
        #recalls.append(recall)
        
        avg_precisions_sum.append(avg_precision)
        
        dict_to_save['precision'] = precision
        #dict_to_save['recall'] = recall
        dict_to_save['avg_precision'] = avg_precision
        with open(path, 'a') as output_file:
            output_file.write(json.dumps(dict_to_save) + '\n')
        if (i+1) % 500 == 0:
            print(f"Analyzed {i+1} queries")
            print("Actual Results...")
            print(f"Mean precision = {np.mean(precisions_sum):.3f}")
            #print(f"Mean recall = {np.mean(recalls):.3f}")
            print(f"Mean Average Precision = {np.mean(avg_precisions_sum):.3f}")
            print(f"Mean Time needed to execute a query = {np.mean(queries_time):.3f}")
    print("FINAL RESULTS ")
    print(f"Mean precision = {np.mean(precisions_sum):.3f}")
    #print(f"Mean recall = {np.mean(recalls):.3f}")
    print(f"Mean Average Precision = {np.mean(avg_precisions_sum):.3f}")
    print(f"Mean Time needed to execute a query = {np.mean(queries_time):.3f}")
    print('Relevantni',relevant_documents)
    print('Vraćeni',results)
    

In [130]:
data

{'questions': [{'body': 'Is the protein Papilin secreted?',
   'documents': ['http://www.ncbi.nlm.nih.gov/pubmed/3320045',
    'http://www.ncbi.nlm.nih.gov/pubmed/7515725',
    'http://www.ncbi.nlm.nih.gov/pubmed/20805556',
    'http://www.ncbi.nlm.nih.gov/pubmed/19297413',
    'http://www.ncbi.nlm.nih.gov/pubmed/19724244',
    'http://www.ncbi.nlm.nih.gov/pubmed/15094122',
    'http://www.ncbi.nlm.nih.gov/pubmed/12666201',
    'http://www.ncbi.nlm.nih.gov/pubmed/21784067',
    'http://www.ncbi.nlm.nih.gov/pubmed/11076767',
    'http://www.ncbi.nlm.nih.gov/pubmed/15094110'],
   'ideal_answer': ['Yes,  papilin is a secreted protein'],
   'exact_answer': 'yes',
   'type': 'yesno',
   'id': '54e25eaaae9738404b000017',
   'snippets': [{'offsetInBeginSection': 1085,
     'offsetInEndSection': 1307,
     'text': 'Using expression analysis, we identify three genes that are transcriptionally regulated by HLH-2: the protocadherin cdh-3, and two genes encoding secreted extracellular matrix prote

## Result for Lexical Auto-id Stopwords False

In [122]:
evaluation(query_parser,data, query_type="lexical", path = "Eval_results_BioASQ_10_abstracts/lexical_results.json")


FINAL RESULTS 
Mean precision = 0.212
Mean Average Precision = 0.387
Mean Time needed to execute a query = 0.225
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('37552126', 1.82634), ('34811514', 1.0), ('32903585', 0.96468), ('15514168', 0.95311), ('18473913', 0.88323), ('11152495', 0.87631), ('12894230', 0.86355), ('21143121', 0.86165), ('34146121', 0.85881)]


## Result for Lexical Pmid Stopwords False

In [123]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
query_parser_pmid = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=True, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic)

In [68]:
evaluation(query_parser_pmid,data, query_type="lexical", path = "Eval_results_BioASQ_10_abstracts/lexical_results_pmid.json")

FINAL RESULTS 
Mean precision = 0.218
Mean Average Precision = 0.404
Mean Time needed to execute a query = 0.312
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 1.0), ('32903585', 0.967719668958904), ('15514168', 0.947230535488355), ('37552126', 0.914876509921199), ('11152495', 0.8772968157872466), ('18473913', 0.8726219844423809), ('12894230', 0.8592261693078889), ('27833949', 0.8573171214184112), ('34146121', 0.8571585020251682), ('21143121', 0.854840207293013)]


## Result lexical pmid with stopwords

In [124]:
english_stopwords = set(stopwords.words('english'))
print(english_stopwords)

{'theirs', 'whom', 'against', 'out', 'we', 'after', 'some', 'couldn', 'her', 'too', 'only', 'his', 'me', "didn't", 'both', 'if', 'yourself', 'hadn', 'won', 'ma', 'isn', 'she', 'further', 'yours', 't', 'themselves', 'needn', 'where', 'shouldn', "wouldn't", 'this', 'but', 'above', 'itself', 'not', 'wasn', 'what', "haven't", 'here', 'again', 'don', 'didn', 'shan', 'they', "shan't", 'how', 'd', "aren't", 'once', 'have', 'those', 'any', 'the', 'own', "needn't", 've', 'more', 'him', 'was', 'at', 'about', 'when', "hasn't", 'weren', 'nor', 'through', 'just', 'on', 'it', 'm', 'an', 'were', 'other', 'into', 'he', "you'd", 'that', "she's", 'mightn', 're', 'hers', 'aren', 'himself', 'haven', 'for', 'or', 'over', 'with', "don't", 'until', 'my', 'as', 'having', 'which', 'hasn', 'ourselves', 'i', 'does', "couldn't", 'most', 'is', 'ours', 'doesn', 'herself', "that'll", 'of', 'then', 'them', 'your', 'in', "you're", 'am', 'ain', "mightn't", 'there', 'wouldn', 's', 'than', 'yourselves', 'should', 'becaus

In [125]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)


In [71]:
evaluation(query_parser_stopwords,data, query_type="lexical", path = "Eval_results_BioASQ_10_abstracts/lex_results_stopwords.json",stopwords_preprocessing=True)

FINAL RESULTS 
Mean precision = 0.226
Mean Average Precision = 0.418
Mean Time needed to execute a query = 0.171
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 1.0), ('15514168', 0.9471752537904271), ('37552126', 0.9150419477774727), ('32903585', 0.9036914396823411), ('11152495', 0.877323588313823), ('12894230', 0.8590731661838458), ('34146121', 0.8570739124739399), ('21143121', 0.8547033136881785), ('30579871', 0.8537225497965703), ('37349855', 0.8355345756465075)]


## Result lexical autoid stopword True

In [72]:
index_name_lexical = "medline-faiss-hnsw-lexical"
lexical_pmid = False
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
evaluation(query_parser_stopwords,data, query_type="lexical", path = "Eval_results_BioASQ_10_abstracts/lex_results_stopwords_auto_id.json",stopwords_preprocessing=True)

FINAL RESULTS 
Mean precision = 0.213
Mean Average Precision = 0.406
Mean Time needed to execute a query = 0.172
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('37552126', 1.8272), ('34811514', 1.0), ('15514168', 0.95297), ('32903585', 0.89945), ('11152495', 0.87637), ('12894230', 0.86315), ('21143121', 0.8613), ('34146121', 0.85859), ('30579871', 0.85806)]


## Result for Semantic without rescore

In [73]:
evaluation(query_parser,data, query_type="semantic", path = "Eval_results_BioASQ_10_abstracts/semantic_results.json")

FINAL RESULTS 
Mean precision = 0.129
Mean Average Precision = 0.322
Mean Time needed to execute a query = 0.208
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('32043568', 1.0), ('11532614', 0.99887), ('15528994', 0.99645), ('17471235', 0.99491), ('26500059', 0.99216), ('11960010', 0.99027), ('31857848', 0.98869), ('32934219', 0.98808), ('31695056', 0.98789), ('34316683', 0.98703)]


# Result Semantic with rescore

In [74]:
query_parser.set_rescore(True)
evaluation(query_parser,data, query_type="semantic", path = "Eval_results_BioASQ_10_abstracts/semantic_results.json")
query_parser.set_rescore(False) # re insert the rescore to False

FINAL RESULTS 
Mean precision = 0.130
Mean Average Precision = 0.327
Mean Time needed to execute a query = 0.218
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('17471235', 1.0), ('11532614', 0.9998), ('26500059', 0.99703), ('15528994', 0.99474), ('32043568', 0.99391), ('11960010', 0.99228), ('19065266', 0.99096), ('26295956', 0.99073), ('31857848', 0.9901), ('31695056', 0.98878)]


## Result for Hybrid with lexical autoid Stopwords False

In [75]:
evaluation(query_parser, data, query_type="hybrid", path = "Eval_results_BioASQ_10_abstracts/hybrid_results.json")

FINAL RESULTS 
Mean precision = 0.163
Mean Average Precision = 0.392
Mean Time needed to execute a query = 0.468
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('37552126', 0.91317), ('34811514', 0.5), ('32043568', 0.5), ('11532614', 0.499435), ('15528994', 0.498225), ('17471235', 0.497455), ('26500059', 0.49608), ('11960010', 0.495135), ('31857848', 0.494345), ('32934219', 0.49404), ('31695056', 0.493945)]


## Result for Hybrid with lexical pmid Stopwords False

In [76]:
evaluation(query_parser_pmid, data, query_type="hybrid", path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid.json")

FINAL RESULTS 
Mean precision = 0.163
Mean Average Precision = 0.415
Mean Time needed to execute a query = 0.460
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.5), ('32043568', 0.5), ('11532614', 0.499435), ('15528994', 0.498225), ('17471235', 0.497455), ('26500059', 0.49608), ('11960010', 0.495135), ('31857848', 0.494345), ('32934219', 0.49404), ('31695056', 0.493945), ('34316683', 0.493515)]


## Result Hybrid pmid with stopwords 

In [126]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.5, beta=0.5, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_stopwords.json", stopwords_preprocessing=True)

FINAL RESULTS 
Mean precision = 0.162
Mean Average Precision = 0.415
Mean Time needed to execute a query = 0.417
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.5), ('32043568', 0.5), ('11532614', 0.499435), ('15528994', 0.498225), ('17471235', 0.497455), ('26500059', 0.49608), ('11960010', 0.495135), ('31857848', 0.494345), ('32934219', 0.49404), ('31695056', 0.493945), ('34316683', 0.493515)]


## Result Hybrid autoid with stopwords 

In [78]:
index_name_lexical = "medline-faiss-hnsw-lexical"
lexical_pmid = False
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.5, beta=0.5, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_autoid_stopwords.json", stopwords_preprocessing=True)

FINAL RESULTS 
Mean precision = 0.164
Mean Average Precision = 0.400
Mean Time needed to execute a query = 0.426
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('37552126', 0.9136), ('34811514', 0.5), ('32043568', 0.5), ('11532614', 0.499435), ('15528994', 0.498225), ('17471235', 0.497455), ('26500059', 0.49608), ('11960010', 0.495135), ('31857848', 0.494345), ('32934219', 0.49404), ('31695056', 0.493945)]


## Result Hybrid pmid with rescore

In [127]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.5, beta=0.5, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.164
Mean Average Precision = 0.410
Mean Time needed to execute a query = 0.414
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.5), ('17471235', 0.5), ('11532614', 0.4999), ('26500059', 0.498515), ('15528994', 0.49737), ('32043568', 0.496955), ('11960010', 0.49614), ('19065266', 0.49548), ('26295956', 0.495365), ('31857848', 0.49505), ('31695056', 0.49439)]


## Result Hybrid 0.6 lexical and 0.4

In [80]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.6, beta=0.4, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_06-04.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.236
Mean Average Precision = 0.423
Mean Time needed to execute a query = 0.406
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.6), ('15514168', 0.5683051522742563), ('37552126', 0.5490251686664837), ('32903585', 0.5422148638094046), ('11152495', 0.5263941529882937), ('12894230', 0.5154438997103075), ('34146121', 0.514244347484364), ('21143121', 0.5128219882129071), ('30579871', 0.5122335298779421), ('37349855', 0.5013207453879045), ('17471235', 0.4)]


## Result Hybrid with alpha 0.7 and Beta 0.3

In [132]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.7, beta=0.3, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_07-03.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.237
Mean Average Precision = 0.427
Mean Time needed to execute a query = 0.522
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.7), ('15514168', 0.6630226776532989), ('37552126', 0.6405293634442308), ('32903585', 0.6325840077776388), ('11152495', 0.614126511819676), ('12894230', 0.601351216328692), ('34146121', 0.5999517387317579), ('21143121', 0.5982923195817249), ('30579871', 0.5976057848575992), ('37349855', 0.5848742029525552), ('17471235', 0.3)]


## Result Hybrid with 0.8 and 0.2

In [82]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.8, beta=0.2, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_08-02.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.237
Mean Average Precision = 0.427
Mean Time needed to execute a query = 0.447
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.8), ('15514168', 0.7577402030323417), ('37552126', 0.7320335582219782), ('32903585', 0.7229531517458729), ('11152495', 0.7018588706510585), ('12894230', 0.6872585329470766), ('34146121', 0.685659129979152), ('21143121', 0.6837626509505429), ('30579871', 0.6829780398372562), ('37349855', 0.668427660517206), ('17471235', 0.2)]


## Result Hybrid with 0.9 lexical and 0.1 semantic

In [83]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.8, beta=0.2, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_09-01.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.237
Mean Average Precision = 0.427
Mean Time needed to execute a query = 0.422
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('34811514', 0.8), ('15514168', 0.7577402030323417), ('37552126', 0.7320335582219782), ('32903585', 0.7229531517458729), ('11152495', 0.7018588706510585), ('12894230', 0.6872585329470766), ('34146121', 0.685659129979152), ('21143121', 0.6837626509505429), ('30579871', 0.6829780398372562), ('37349855', 0.668427660517206), ('17471235', 0.2)]


## Result Hybrid with 0.4 lexical and 0.6 semantic

In [84]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.4, beta=0.6, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_04-06.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.160
Mean Average Precision = 0.359
Mean Time needed to execute a query = 0.425
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('17471235', 0.6), ('11532614', 0.59988), ('26500059', 0.5982179999999999), ('15528994', 0.5968439999999999), ('32043568', 0.5963459999999999), ('11960010', 0.595368), ('19065266', 0.594576), ('26295956', 0.594438), ('31857848', 0.5940599999999999), ('31695056', 0.593268), ('34811514', 0.4)]


## Result Hybrid with 0.3 lexical and 0.7 semantic

In [85]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.3, beta=0.7, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_03-07.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.160
Mean Average Precision = 0.355
Mean Time needed to execute a query = 0.407
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('17471235', 0.7), ('11532614', 0.6998599999999999), ('26500059', 0.6979209999999999), ('15528994', 0.6963179999999999), ('32043568', 0.6957369999999999), ('11960010', 0.694596), ('19065266', 0.693672), ('26295956', 0.693511), ('31857848', 0.69307), ('31695056', 0.6921459999999999), ('34811514', 0.3)]


## Result Hybrid with 0.2 lexical and 0.8 semantic

In [86]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.2, beta=0.8, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_02-08.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.160
Mean Average Precision = 0.357
Mean Time needed to execute a query = 0.420
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('17471235', 0.8), ('11532614', 0.7998400000000001), ('26500059', 0.797624), ('15528994', 0.795792), ('32043568', 0.7951280000000001), ('11960010', 0.7938240000000001), ('19065266', 0.792768), ('26295956', 0.7925840000000001), ('31857848', 0.79208), ('31695056', 0.7910240000000001), ('34811514', 0.2)]


## Result Hybrid with 0.1 lexical and 0.9 semantic

In [87]:
index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True
query_parser_stopwords = QueryProcessor(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic, stopwords=english_stopwords)
query_parser_stopwords.set_rescore(True)
evaluation(query_parser_stopwords, data, query_type="hybrid",alpha=0.1, beta=0.9, path = "Eval_results_BioASQ_10_abstracts/hybrid_results_pmid_recore_01-09.json", stopwords_preprocessing=True)
query_parser_stopwords.set_rescore(False)

FINAL RESULTS 
Mean precision = 0.160
Mean Average Precision = 0.358
Mean Time needed to execute a query = 0.412
Relevantni ['9766517', '31603123', '11532614', '27833949', '34062779', '21200140', '12432243', '35628286', '34605036', '34146121']
Vraćeni [('17471235', 0.9), ('11532614', 0.8998200000000001), ('26500059', 0.897327), ('15528994', 0.895266), ('32043568', 0.894519), ('11960010', 0.8930520000000001), ('19065266', 0.891864), ('26295956', 0.891657), ('31857848', 0.89109), ('31695056', 0.889902), ('34811514', 0.1)]


# Evaluation based on PubMed website

In [89]:
from Bio import Entrez
# Always tell NCBI who you are (your email address)
Entrez.email = "adela.ljajic@ivi.ac.rs"
def search_pubmed(query, limit = 10, mesh=True):
    if not mesh:
        query += "[Title/Abstract]"
    # Use Entrez.esearch to search for articles matching the query in PubMed
    handle = Entrez.esearch(db="pubmed", term=query, retmax=limit, sort="relevance",)
    
    record = Entrez.read(handle)
    handle.close()
    # Get the list of Ids returned by the search
    id_list = record["IdList"]
    return id_list

def fetch_details(id_list):
    # Use Entrez.efetch to get the article details from the list of Ids
    ids = ','.join(id_list)
    handle = Entrez.efetch(db="pubmed", id=ids, retmode="xml")
    records = Entrez.read(handle)
    handle.close()
    return records
# Example usage

query = "Is the protein Papilin secreted?"
id_list = search_pubmed(query)
print("Id list = ",id_list)


Id list =  ['11076767', '38284126', '15094110', '30388611', '38837578', '12666201', '30661986', '22242013', '36997062', '22937083']


In [94]:
query = "Is the protein Papilin secreted?"
id_list = search_pubmed(query, mesh=False)
print("Id list = ",id_list)

Id list =  ['11076767', '38284126', '15094122', '21784067', '7515725']


In [105]:
def average_precision(retrived_doc, true_doc):
    # Initialize variables
    precision_sum = 0
    num_retrieved_docs = 0
    
    # Calculate precision at each relevant document position
    for i, retrived in enumerate(retrived_doc, start=1):
        pmid = retrived  # CHAGEd from pmid, _ = retrived to pmid = retrived (CAUSED PROBLEM IN EVALUATION ON PUBMEDSEARCH)
        if pmid in true_doc:  # Check if the document is relevant
            num_retrieved_docs += 1
            precision_sum += num_retrieved_docs / i  # Calculate precision at cutoff i

    # Calculate average precision
    if num_retrieved_docs == 0:
        return 0
    else:
        return precision_sum / num_retrieved_docs

def evaluation_pubmed(query_type, data,mesh=True,path = "Eval_results_BioASQ_10_abstracts/query_result.json"):
    avg_precisions_sum = [] # sum all average precision and divide with number of queries 
    precisions_sum = []
    queries_time = []
    for i,question in enumerate(data['questions']):
        dict_to_save = {}
        query = question['body']
        dict_to_save['query'] = query
        dict_to_save['query_type'] = query_type
        relevant_documents = clean_documents(question['documents'])
        start_time = time.time()
        
        results = search_pubmed(query, limit = len(relevant_documents),mesh=mesh)
        queries_time.append(time.time() - start_time)
        
        dict_to_save['true_documents'] = list(relevant_documents)
        dict_to_save['retrieved_documents'] = results
       
        number_retrieved_documents = 0
        print(results)
        for pmid in results:
            if pmid in relevant_documents:
                number_retrieved_documents +=1

        precision = number_retrieved_documents / len(relevant_documents)
        recall = number_retrieved_documents / len(relevant_documents)
        avg_precision = average_precision(results, relevant_documents)
       
        precisions_sum.append(precision)
        #recalls.append(recall)
        
        avg_precisions_sum.append(avg_precision)
        
        dict_to_save['precision'] = precision
        #dict_to_save['recall'] = recall
        dict_to_save['avg_precision'] = avg_precision
        with open(path, 'a') as output_file:
            output_file.write(json.dumps(dict_to_save) + '\n')
        if (i+1) % 500 == 0:
            print(f"Analyzed {i+1} queries")
            print("Actual Results...")
            print(f"Mean precision = {np.mean(precisions_sum):.3f}")
            #print(f"Mean recall = {np.mean(recalls):.3f}")
            print(f"Mean Average Precision = {np.mean(avg_precisions_sum):.3f}")
            print(f"Mean Time needed to execute a query = {np.mean(queries_time):.3f}")
    print("FINAL RESULTS ")
    print(f"Mean precision = {np.mean(precisions_sum):.3f}")
    #print(f"Mean recall = {np.mean(recalls):.3f}")
    print(f"Mean Average Precision = {np.mean(avg_precisions_sum):.3f}")
    print(f"Mean Time needed to execute a query = {np.mean(queries_time):.3f}")

## Evaluation Pubmed with Mesh Term

The mesh terms are applied automatically by PubMed website

In [106]:
evaluation_pubmed(data=data, query_type="PubMed website", path="Eval_results_BioASQ_subset/Pubmed_mesh.json")

['11076767', '38284126', '15094110', '30388611', '38837578', '12666201', '30661986', '22242013', '36997062', '22937083']
['30515780', '34024716', '36028118', '32092478', '29635231', '31545469', '33114380', '33893301', '29458123', '32193744']
[]
['22094949', '25618141', '36622962', '34843880', '23393190', '35015757', '23748105', '30607432', '25207863', '24344320']
['21292530']
['35582186', '21358698', '25230708', '33244632', '31109940', '36742404', '24529566', '28500396', '19114612', '22913895']
['37057884', '29757017', '33460272', '30069763', '32673417', '37170083', '37998388', '26847859', '34696930', '25659402']
['29899761', '16157025', '30031766', '10379685', '37686475', '10571517', '20442286', '26477352', '27351911', '30474100']
['30247122', '38607052']
['35302338', '779595', '39194734', '31663791', '33326012', '28082330', '21484537', '33339418', '23473408', '3779737']
[]
['26074382', '25201530', '22722839', '37020028', '32220301', '30527665', '29858674', '34408788', '34380014', '23

## Evaluation PubMed without mesh

In [107]:
evaluation_pubmed(data=data, query_type="PubMed website no mash", mesh = False, path="Eval_results_BioASQ_subset/Pubmed_no_mesh.json")

['11076767', '38284126', '15094122', '21784067', '7515725']
['31431733', '36028118', '27906049', '26925062', '27105133', '31486093', '22465238', '33308247', '31763801', '22360322']
[]
['22094949', '25618141', '34843880', '36622962', '35015757', '23748105', '30607432', '24344320', '23393190', '25207863']
['21292530']
[]
['36880482']
['29899761', '16157025', '30474100', '9496833', '21063743']
['38607052']
['39194734', '28082330', '33339418', '20875630', '20632954', '12643345', '21329899', '33191933', '35915685', '30732632']
[]
['30079888', '28332978', '30068710', '27170440']
['22100391', '11566103', '23679081', '8910287', '15888437', '9030581', '23159405']
['33925995']
[]
[]
['30633901', '38442890', '28596378', '35143849', '20116986', '37298433', '21612942', '30642153', '37019879', '26738338']
['27398650', '36856617', '30753825', '25891173', '30675668', '27433843', '29658430', '35867951', '30715153', '35367007']
['27066913', '28904069', '35643636', '1524556']
[]
['28887948', '29447987', 

## Create json file for BioASQ questions with our retrieval results - for 182 question from BioASQ (contains exactly 10 relevant abstracts)
#### Answer on BioASQ questions using our IR (0,7 0,3 combination) - look down after client initialization



In [34]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd



# Create new query processor for extracting data for generative component
class QueryProcessor_Generative:
    def __init__(self, index_lexical:str = "medline-faiss-hnsw-lexical",lexical_pmid = False, index_name_semantic ="medline-faiss-hnsw", rescore = False, model=None, lexical_client=None, semantic_client=None, output_file_path="queries/queries.tsv", stopwords=set([])):
        self.index_lexical_name = index_lexical
        self.index_name_semantic = index_name_semantic
        # 2 index name (?)
        self.model = model
        #self.lexical_pmid = lexical_pmid
        self.lexical_client = lexical_client
        self.semantic_client = semantic_client
        self.output_file_path = output_file_path
        self.stop_words = stopwords
        self.query_result = []
        self.rescore = rescore
        self.lexical_query = self.lexical_query_pmid if lexical_pmid else self.lexical_query
    
    def set_rescore(self, rescore):
        self.rescore = rescore

    def preprocess_query(self, query_str):
        return ' '.join([word for word in word_tokenize(query_str) if word.lower() not in self.stop_words])

    def save_results(self):
        with open(self.output_file_path, "w") as file:
            json.dump(self.query_result, file, indent=4)
      
    
    def reorder_pmid(self, retrived_documents):
        pmid_scores = {}
        
        # Iterate through the set data
        for _, value in retrived_documents.items():
            pmid = value['pmid']
            score = value['score']
            
            # Check if pmid already exists in the dictionary
            if pmid in pmid_scores:
                pmid_scores[pmid] += score
            else:
                pmid_scores[pmid] = score
           
        return pmid_scores
    
    def lexical_query(self, query_str, limit=10):
        if self.lexical_client == None:
            raise ValueError("No Lexical client defined")
        
        query = {
                "size": limit,
                "query": {
                    "multi_match": {
                        "query": query_str,
                        "fields": ["text"]
                    }
                }
            }
       
        results = self.lexical_client.search(index=self.index_lexical_name, body=query) 
        retrived_documents = {}
        max_score = results['hits']['max_score']
     
        for hit in results["hits"]["hits"]:
            
            pmid = hit["_source"]["pmid"]
            score = hit["_score"]
            auto_id = hit["_id"]
            
            
            
            retrived_documents[auto_id] = {
                "score": round(score/max_score, 5),
                "pmid": pmid
                }
        
        retrived_documents = self.reorder_pmid(retrived_documents)
        return retrived_documents #adjust the return 
    
    def lexical_query_pmid(self, query_str, limit=10):
        #print("Lexical = ",query_str)
        if self.lexical_client == None:
            raise ValueError("No Lexical client defined")
        
        query = {
                "size": limit,
                "query": {
                    "multi_match": {
                        "query": query_str,
                        "fields": ["full_text"]
                    }
                }
            }
        
        results = self.lexical_client.search(index=self.index_lexical_name, body=query) 
        
        retrieved_documents = {}
        max_score = results['hits']['max_score']
        for hit in results["hits"]["hits"]:
            
            pmid = hit["_source"]["pmid"]
            score = hit["_score"] / max_score
            
            retrieved_documents[pmid] = score
            
        return retrieved_documents #adjust the return 

    def semantic_query(self, query, limit=10):
        #print("semantic = ",query)
        if self.semantic_client == None:
            raise ValueError("No Semantic client defined")
        if self.model == None:
            raise ValueError("No model defined")
        
        query_vector = self.model.encode(query).tolist()
    
        search_params=models.SearchParams(
            quantization=models.QuantizationSearchParams(rescore=self.rescore)
            )
        results = self.semantic_client.search(collection_name=self.index_name_semantic,query_vector=query_vector,search_params=search_params, limit=limit)
    
        #results = self.semantic_client.search(collection_name=self.index_name_semantic,query_vector=query_vector, limit=limit)
        
        retrived_documents = {}
        max_score = None
        for i,document in enumerate(results):
            
            pmid = document.payload['pmid']
            score = document.score
            if i == 0:
                # first score is the max
                max_score = score
            retrived_documents[document.id] = { 'pmid': pmid, 'score': round(score / max_score, 5) } 

        retrived_documents = self.reorder_pmid(retrived_documents)
        
        return retrived_documents
    

    def hybrid_query(self, query_lexical, query_semantic, lex_parameter = 0.7, semantic_parameter = 0.3, limit=10):
        if (lex_parameter + semantic_parameter) > 1:
            raise ValueError("Uncorrect parameters for Hybrid Queries")
        #changed for hybrid (also need to be changed in general Query processor)
        lexical_results = self.lexical_query(query_lexical, limit = limit) 
        semantic_results = self.semantic_query(query_semantic, limit)
        max_score = 0
        retrived_documents = {}
        
        for lex_pmid in lexical_results:
            score = lexical_results[lex_pmid] * lex_parameter
            if lex_pmid in semantic_results:
                score += semantic_results[lex_pmid] * semantic_parameter

            retrived_documents[lex_pmid] = score
            max_score = max(max_score, score)
            

        for semantic_pmid in semantic_results:
            if semantic_pmid not in lexical_results:
                score = semantic_results[semantic_pmid] * semantic_parameter
                retrived_documents[semantic_pmid] = score
                max_score = max(max_score, score)
                
        return retrived_documents # just to have a starting point

 #changed line to retrieve exact number of documents (to return full_text), added return statement for the same reason
    def execute_query(self, query_str, query_type='lexical', lex_parameter=0.7, semantic_parameter=0.3, limit=10, save=True, stopwords_preprocessing=True):
        text_query = self.preprocess_query(query_str) if stopwords_preprocessing else query_str

        if query_type == 'lexical':
            results = self.lexical_query(text_query, limit=limit)
        elif query_type == 'semantic':
            results = self.semantic_query(query_str, limit=limit)
        elif query_type == 'hybrid':
            results = self.hybrid_query(text_query, query_str, lex_parameter, semantic_parameter, limit=limit)
        else:
            raise ValueError("Invalid query type specified. Choose 'lexical', 'semantic', or 'hybrid'.")

        document_retrieved = sorted(results.items(), key=lambda x: x[1], reverse=True)
        document_retrieved = document_retrieved[:limit]  # Limit the number of documents

        if save:
            return self.process_results(document_retrieved, query_str, query_type)

        # Optionally return the documents if not saving
        return document_retrieved


 

# changed to retrieve full_text (for creating custom dataset Bioasq question and our IR's retrieved abstracts)
    def process_results(self, results, query_str, query_type):
        retrieved_documents = []
        for element in results:
            pmid, _ = element
            query = {
                "query": {
                    "term": {
                        "pmid": int(pmid)
                    }
                }
            }

            # Fetching the document by pmid to get the full_text
            results = self.lexical_client.search(index=self.index_lexical_name, body=query)
            if results['hits']['hits']:
                full_text = results['hits']['hits'][0]["_source"]['full_text']
                full_text=' '.join(full_text.replace('\n', ' ').split()) # remoce newlines
                pmid = results['hits']['hits'][0]["_source"]['pmid']

                retrieved_documents.append({
                    "pmid": pmid,
                    "full_text": full_text
                })

        dict_to_save = {'query': query_str, 'query_type': query_type, 'abstracts': retrieved_documents}
        self.query_result.append(dict_to_save)
        return retrieved_documents
            

        



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
import json


index_name_lexical = "medline-faiss-hnsw-lexical-pmid"
lexical_pmid = True

model_card = 'sentence-transformers/msmarco-distilbert-base-tas-b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device {device}")

model=SentenceTransformer(model_card)

Device cpu


In [36]:
known_pmid = '15795476'
known_pmid = '21995290'
print(get_document_text(known_pmid, client_lexical))

Comprehensive analysis of RET common and rare variants in a series of Spanish Hirschsprung patients confirms a synergistic effect of both kinds of events. BACKGROUND RET is the major gene associated to Hirschsprung disease (HSCR) with differential contributions of its rare and common, coding and noncoding mutations to the multifactorial nature of this pathology. In the present study, we have performed a comprehensive study of our HSCR series evaluating the involvement of both RET rare variants (RVs) and common variants (CVs) in the context of the disease. METHODS RET mutational screening was performed by dHPLC and direct sequencing for the identification of RVs. In addition Taqman technology was applied for the genotyping of 3 RET CVs previously associated to HSCR, including a variant lying in an enhancer domain within RET intron 1 (rs2435357). Statistical analyses were performed using the SPSS v.17.0 to analyze the distribution of the variants. RESULTS Our results confirm the stronges

In [37]:
lexical_pmid = False

if lexical_pmid:
    index_name_lexical = 'medline-faiss-hnsw-lexical-pmid'
else:
    index_name_lexical ='medline-faiss-hnsw-lexical'

coll_name_semantic = "medline-faiss-hnsw"

query_parser_generative = QueryProcessor_Generative(index_lexical=index_name_lexical, lexical_pmid=lexical_pmid, index_name_semantic = coll_name_semantic, model= model, lexical_client=client_lexical, semantic_client=client_semantic)

In [38]:
query_parser_generative.execute_query(query_str="Which gene is responsible for disfunction in speech for children?", query_type='hybrid')

[{'pmid': '10797929',
  'full_text': 'Opercular epileptic syndrome: an unusual form of benign partial epilepsy in childhood. INTRODUCTION Functional opercular syndrome in childhood is an exceptional form of presentation of benign partial epilepsy with centro-temporal rolandic spikes (BECRS). CLINICAL CASES We studied the evolution of four patients, three of them followed for more than 15 years. Two were siblings, and their father suffered from BECRS with permanent language problems (verbal dyspraxia) and difficulty of protunding his tongue in adulthood. A third patient suffered benign familial neonatal convulsions (BFNC). In all four patients the actual illness begun as a BECRS with opercular troubles as an ictal phenomena. At about four years of age, the opercular disfunction became evident, with severe drooling, facial hypomobility and speech disturbance which waxed and vanished along weeks, months or years, apparently not ictal. Antiepileptic drugs not only were unable to control th

In [39]:
english_stopwords = set(stopwords.words('english'))

def query_ir_system(query_str, query_type='hybrid', lex_parameter=0.7, semantic_parameter=0.3, limit=10):
    # You would need to initialize the QueryProcessor with actual parameters including the model and clients
    qp = QueryProcessor_Generative(
        model=model,  
        lexical_client=client_lexical,  
        semantic_client=client_semantic  
    )
    
    # Execute the query using the hybrid method by default
    results = qp.execute_query(query_str, query_type=query_type, lex_parameter=lex_parameter, semantic_parameter=semantic_parameter, limit=limit)
    #print(results)
    #print('/n')
    # Convert results to the required format
    #abstracts = [{'id': pmid, 'text': text} for pmid, text in results]
    abstracts = [{'id': pmid, 'text': full_text, 'score': score} for pmid, full_text, score in results]

    #abstracts = [{'id': result['pmid'], 'text': result['text']} for result in results]

    return abstracts

def process_questions(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    results = {'questions': []}
    
    for question in data['questions']:
        question_text = question['body']
        question_id = question['id']
        question_type = question.get('type', 'summary')  # Assume 'summary' if not specified
        
        # Query the IR system to get abstracts
        abstracts = query_parser_generative.execute_query(question_text,lex_parameter=0.7, semantic_parameter=0.3,limit=10, query_type='hybrid') 
        
        results['questions'].append({
            'id': question_id,
            'question': question_text,
            'type': question_type,
            'abstracts': abstracts
        })
    
    return results





In [40]:
# Path to the file containing the questions

file_path = 'BioASQ_10_abstracts.json'

# Process the questions and retrieve abstracts
output_data = process_questions(file_path)

# Save the output data to a JSON file
output_file_path = 'Generative/BioASQ_10_abstracts-AbbrsractFromOurIR.json'
with open(output_file_path, 'w') as output_file:
    json.dump(output_data, output_file, indent=4)

print(f"Output written to {output_file_path}")

Output written to Generative/BioASQ_10_abstracts-AbbrsractFromOurIR.json


In [41]:
known_pmid = '15795476'
qp = QueryProcessor_Generative(
        model=model,  # You need to define or load your model
        lexical_client=client_lexical,  # You need to define your ElasticSearch client or equivalent for lexical queries
        semantic_client=client_semantic  # Define your semantic search client
    )
print(qp.execute_query("What is the cause of hip pain",query_type='hybrid',lex_parameter=0.7, semantic_parameter=0.3,limit=10))

[{'pmid': '31431276', 'full_text': 'Current trends in sport and exercise hip conditions: Intra-articular and extra-articular hip pain, with detailed focus on femoroacetabular impingement (FAI) syndrome. Musculoskeletal conditions, such as hip pain are leading causes of pain and disability. Hip pain is the second most common cause of lower limb musculoskeletal pain, and is commonly seen in active individuals. Hip and groin pain may have intra-articular and extra-articular causes. Femoroacetabular impingement (FAI) syndrome and the associated pathologies are common intra-articular causes of hip and groin pain in active individuals. There are also a number of extra-articular causes of pain, which include musculotendinous conditions, extra-articular impingements and the clinical entities of groin pain described in the Doha agreement. This chapter will describe these, with a detailed focus on FAI syndrome. Specifically, it addresses: 1. What is and what causes FAI syndrome; 2. How do I diag

In [42]:
#print(qp.hybrid_query(client_lexical,client_semantic,lex_parameter=0.7,semantic_parameter=0.3,limit=10))
print(qp.lexical_query("What is the cause of hip pain",limit=10))

{'31431276': 1.0, '1570146': 0.9046, '26692249': 0.90139, '30509429': 0.89442, '37751184': 0.88643, '37777849': 0.88643, '33132093': 0.86732, '31272626': 0.84404, '36621448': 0.84001, '1622374': 0.83702}


In [43]:
known_pmid = '15795476'
print(query_ir_system("What is the cause of hip pain", query_type='hybrid', lex_parameter=0.7, semantic_parameter=0.3, limit=10))

ValueError: not enough values to unpack (expected 3, got 2)